In [43]:
import pandas as pd
df = pd.read_csv(f'data_storage/stock_price_data/Energy/APA.csv')

In [44]:
df['Date'] = pd.to_datetime(df['Date'])

In [45]:
df[df['Date'] == pd.to_datetime("07/01/2024")].index[0]

106

In [46]:
df.sort_values(by="Date").reset_index()

,index,Date,Close/Last,Volume,Open,High,Low
0,1257,2019-12-02,$19.54,22790570,$20.00,$20.42,$18.93
1,1256,2019-12-03,$18.38,15187360,$19.25,$19.30,$18.33
2,1255,2019-12-04,$18.70,10408040,$18.61,$19.16,$18.5003
3,1254,2019-12-05,$18.58,7503783,$18.78,$18.9824,$18.44
4,1253,2019-12-06,$19.99,10085920,$18.50,$20.17,$18.33
...,...,...,...,...,...,...,...
1253,4,2024-11-22,$22.77,12821090,$22.29,$22.99,$22.265
1254,3,2024-11-25,$22.67,7440440,$22.79,$22.98,$22.435
1255,2,2024-11-26,$22.52,4680421,$22.76,$22.835,$22.245
1256,1,2024-11-27,$22.52,4679593,$22.54,$23.14,$22.36


In [103]:
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(by="Date").reset_index()
date = pd.to_datetime("07/01/2024")
while len(df[df['Date'] == date]) == 0:
    date += pd.DateOffset(days=1)
index = df[df['Date']==pd.to_datetime(date)].index[0]
output_index = df[df['Date']==pd.to_datetime(date)+pd.DateOffset(days=30)].index[0]


In [113]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np


features = ['Open', 'Low', 'Volume']

num_data = df[['Open','High','Low']].to_numpy().astype(str)
num_data = np.char.replace(num_data, '$', '')


output_var = df[['Close/Last']].to_numpy().astype(str)
output_var = np.char.replace(output_var, '$', '')
output_var = output_var.astype(float)

scaler = MinMaxScaler()
feature_transform = scaler.fit_transform(num_data)
feature_transform= pd.DataFrame(columns=features, data=feature_transform, index=df.index)

X_train, X_test = feature_transform[:index], feature_transform[index:]
offset = len(output_var[:output_index]) - len(output_var[:index])
y_train, y_test = output_var[offset:output_index], output_var[output_index:]

In [114]:
y_train

array([[25.36],
       [25.69],
       [25.64],
       ...,
       [31.21],
       [30.78],
       [31.03]])

In [115]:
trainX =np.array(X_train)
testX =np.array(X_test)
X_train = trainX.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test = testX.reshape(X_test.shape[0], 1, X_test.shape[1])

In [116]:
X_train.shape

(1151, 1, 3)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.layers import LSTM
from keras.utils import plot_model

# Define early stopping
early_stop = EarlyStopping(
    monitor='loss',   # Monitor validation loss
    patience=5,           # Stop after 3 epochs with no improvement
    restore_best_weights=True  # Roll back to best model
)

lstm = Sequential()
lstm.add(LSTM(32, input_shape=(1, trainX.shape[1]), activation='relu', return_sequences=False))
lstm.add(Dense(1))
lstm.compile(loss='mean_squared_error', optimizer='adam')
plot_model(lstm, show_shapes=True, show_layer_names=True)
history=lstm.fit(X_train, y_train, epochs=30, batch_size=8, verbose=1, shuffle=False, callbacks=[early_stop])


You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.
Epoch 1/30
144/144 [==============================] - 2s 2ms/step - loss: 918.3852
Epoch 2/30
144/144 [==============================] - 0s 2ms/step - loss: 791.4161
Epoch 3/30
144/144 [==============================] - 0s 2ms/step - loss: 542.4244
Epoch 4/30
144/144 [==============================] - 0s 2ms/step - loss: 271.7872
Epoch 5/30
144/144 [==============================] - 0s 2ms/step - loss: 101.5590
Epoch 6/30
144/144 [==============================] - 0s 2ms/step - loss: 39.8326
Epoch 7/30
144/144 [==============================] - 0s 2ms/step - loss: 27.2368
Epoch 8/30
144/144 [==============================] - 0s 2ms/step - loss: 25.7645
Epoch 9/30
144/144 [==============================] - 0s 2ms/step - loss: 25.6804
Epoch 10/30
144/144 [==============================] - 0s 2ms/step - loss: 25.6936
Epoch 11/30
144/144 [====